In [1]:
!export WANDB_API_KEY="4a8fc5b45c6c79237bcb32a9659c090b1458dde7"
!pip install --quiet wandb
!pip install --quiet nvidia-dali-cuda110
!pip install --quiet lru-dict
!pip install --quiet efficientnet_pytorch
!pip install --quiet pytorch_lightning

In [16]:
import wandb
import torch
import os
import numpy as np
import json
from PIL import Image
from training_wheels import TrainingWheels
from dataset import DataloopDataset, DataloopFiles
from augmentation import Augmentation

# Checking model performance on light dirt vehicles

In [12]:
hparams = {
    'dataset_dir':"dataloop",
    'model':"resnet18",
    'use_wandb':True,
    'enable_random_cropping':False,
    'batch_size':2,
    'num_gpus':1,
    'downscaling_width':1224,
    'downscaling_height':1632,
    'max_epochs':50,
    'accelerator':None,
    'devices':None,
    'use_dali':False,
    'center_crop':448,
    'enable_vertical_mirroring':False,
    'enable_horizontal_mirroring':True,
    'random_rotation_angle':15,
    'noise_amount':0,
    'resume_from_checkpoint':None,
    'enable_image_logging':True,
    'lr':0.00003,
    'balance_sampler':False,
    'train_sample_size':250
}

In [13]:
# This loads the augmentation object (see the keywords and how they are used inside )
augmentation = Augmentation(enable_random_cropping=hparams['enable_random_cropping'],
                            enable_vertical_mirroring=hparams['enable_vertical_mirroring'],
                            enable_horizontal_mirroring=hparams['enable_horizontal_mirroring'],
                            random_rotation_angle=hparams['random_rotation_angle'],
                            noise_amount=hparams['noise_amount'],
                            downscaling_width=hparams['downscaling_width'],
                            downscaling_height=hparams['downscaling_height'])

In [4]:
target_dir = hparams['dataset_dir']

In [5]:
run = wandb.init()
artifact = run.use_artifact('fyusion/car-condition-classifier/model-c4d6x0p0:v49', type='model')
artifact_dir = artifact.download()
model = TrainingWheels.load_from_checkpoint(artifact_dir+"/model.ckpt")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ashley-lawrencehuizenga (fyusion). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-c4d6x0p0:v49, 128.86MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [6]:
# This loads the Validation images. Note currently it's set to a specific seed so the loading should be the same each run...
model.setup()

In [7]:
# This saves the file list. Unfortunately, the file_names can't be grabbed unless we actually grab the image itself, 
# which takes a while
file_list = []
for val in iter(model.validation_data):
    file_list+=[val[0]]

In [8]:
# Getting only light dirt images
class_to_label = {"clean": 0, "light_dirt": 1, "heavy_dirt": 0}
files = []
labels = []
file_names = []

for file in file_list:
    try:
        target_file = os.path.join(target_dir, "items", file)
        Image.open(target_file)
        # Try to open the json file
        annotation_file_name = os.path.join(
            target_dir, "json", os.path.splitext(file)[0] + ".json"
        )
        with open(annotation_file_name) as annotation_file:
            json_data = json.load(annotation_file)
            img_width = json_data["metadata"]["system"]["width"]
            img_height = json_data["metadata"]["system"]["height"]
            annotations = []
            for annotation in json_data["annotations"]:
                label = annotation["label"]
                try:
                    coords = annotation["coordinates"]
                    p1 = np.array([coords[0]["x"], coords[0]["y"]])
                    p2 = np.array([coords[1]["x"], coords[1]["y"]])
                    bbox_size = np.linalg.norm(p2 - p1)
                    annotations.append(
                        [bbox_size, class_to_label[label], coords]
                    )
                except KeyError:
                    try:
                        annotations.append([1, class_to_label[label], 0])
                    except KeyError:
                        continue
            annotations = sorted(annotations, key=lambda x: x[0], reverse=True)
            # Only the biggest bounding box is considered
            if len(annotations) == 0:
                continue
            if annotations[0][1] == 1:
                files.append(target_file)
                labels.append(annotations[0][1])
                file_names.append(file)
    except FileNotFoundError:
        # We just ignore files that are not images or do not have a json file
        continue
assert len(files) == len(labels)

In [24]:
# This is specifically written to pull all files in a list
valid_set = DataloopFiles(dataset_dir='dataloop',file_list=file_names,
                          train=False,
                          augmentation=augmentation)

In [25]:
# This takes a while, loading an image and scoring an image both happen here
# Note the need for no_grad. If this isn't called memory usage goes through the roof quickly
output_valid = []
for val in valid_set:
    with torch.no_grad():
        temp = model(val[1].unsqueeze(0).to('cuda'))
    output_valid.append(temp)
    del temp
    torch.cuda.empty_cache()

In [27]:
# This bit of code checks whether the Clean Val is larger than Dirt value
list_val = []
for j in output_valid:
    temp = j.cpu().numpy()[0]
    list_val.append(temp[0]>temp[1])
    del temp

In [28]:
list_val

[True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True]

In [29]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
